In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Input and Output Features
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [6]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [7]:
X.shape

(768, 8)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [10]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.fit(X_train,y_train, batch_size=32, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5747 - loss: 0.6919 - val_accuracy: 0.5584 - val_loss: 0.7054
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6586 - loss: 0.6313 - val_accuracy: 0.6558 - val_loss: 0.6564
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7147 - loss: 0.5958 - val_accuracy: 0.7208 - val_loss: 0.6181
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7273 - loss: 0.5564 - val_accuracy: 0.7273 - val_loss: 0.5871
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7778 - loss: 0.5211 - val_accuracy: 0.7208 - val_loss: 0.5623
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7347 - loss: 0.5316 - val_accuracy: 0.7208 - val_loss: 0.5405
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7748 - loss: 0.5013 - val_accuracy: 0.7403 - val_loss: 0.5225
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7447 - loss: 0.5258 - val_accuracy: 0.7468 - v

In [12]:
pip install -U keras-tuner

In [13]:
import keras_tuner as kt

In [14]:
def build_model(hp):
    """
    Builds a Keras model with tunable hyperparameters.

    Args:
        hp: A HyperParameters object from Keras Tuner.

    Returns:
        A compiled Keras model.
    """
    from keras import layers
    from keras import regularizers

    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],))) # Input layer

    # Tune whether to use regularization
    use_regularization = hp.Boolean('use_regularization')
    kernel_regularizer = None
    if use_regularization:
        # Tune the type of regularizer (L1 or L2)
        regularizer_type = hp.Choice('regularizer_type', values=['l1', 'l2'])
        # Tune the strength of the regularizer (logarithmic scale)
        regularizer_strength = hp.Float('regularizer_strength', min_value=1e-5, max_value=1e-2, sampling='log')

        if regularizer_type == 'l1':
            kernel_regularizer = regularizers.l1(regularizer_strength)
        else: # 'l2'
            kernel_regularizer = regularizers.l2(regularizer_strength)

    # Tune the number of hidden layers
    for i in range(hp.Int('num_hidden_layers', min_value=1, max_value=5, step=1)):
        # Tune the number of units in each hidden layer
        model.add(layers.Dense(units=hp.Int(f'units_layer_{i}', min_value=32, max_value=512, step=32),
                                activation=hp.Choice(f'activation_layer_{i}', values=['relu', 'tanh', 'sigmoid']),
                                kernel_regularizer=kernel_regularizer)) # Apply regularization here
        # Add dropout layer
        model.add(layers.Dropout(hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Tune the optimizer
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])
    # Tune the learning rate for the chosen optimizer
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='log')

    if optimizer_choice == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    else: # 'rmsprop'
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [15]:
print("\nInstantiating Keras Tuner (RandomSearch)...")
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy', # Optimize for validation accuracy
    max_trials=50,             # Try 50 different hyperparameter combinations
    executions_per_trial=2,    # Train each model 2 times to reduce variance
    directory='keras_tuner_dir', # Directory to save results
    project_name='ann_tuning'    # Project name
)


Instantiating Keras Tuner (RandomSearch)...


In [16]:
# Print a summary of the search space
print("\nSearch space summary:")
tuner.search_space_summary()


Search space summary:
Search space summary
Default search space size: 7
use_regularization (Boolean)
{'default': False, 'conditions': []}
num_hidden_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_layer_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation_layer_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rmsprop'], 'ordered': False}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [17]:
# Run the Hyperparameter Search
print("\nStarting hyperparameter search...")
# The search method will train models for different hyperparameter combinations.
# It uses the training and validation data provided.
tuner.search(X_train, y_train,
             epochs=20,          # Number of epochs for each trial
             validation_data=(X_test, y_test),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]) # Stop if val_loss doesn't improve

print("\nHyperparameter search complete.")

Trial 50 Complete [00h 00m 29s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.8019480407238007
Total elapsed time: 00h 13m 29s

Hyperparameter search complete.


In [18]:
# Get the Best Model(s)
# Display a summary of the best trials found
print("\nBest trials summary:")
tuner.results_summary()


Best trials summary:
Results summary
Results in keras_tuner_dir/ann_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 47 summary
Hyperparameters:
use_regularization: True
num_hidden_layers: 2
units_layer_0: 128
activation_layer_0: tanh
dropout_0: 0.30000000000000004
optimizer: rmsprop
learning_rate: 0.0007569660497386584
regularizer_type: l2
regularizer_strength: 0.00014583963087498662
units_layer_1: 128
activation_layer_1: relu
dropout_1: 0.2
units_layer_2: 256
activation_layer_2: tanh
dropout_2: 0.4
units_layer_3: 384
activation_layer_3: sigmoid
dropout_3: 0.2
units_layer_4: 512
activation_layer_4: tanh
dropout_4: 0.30000000000000004
Score: 0.8019480407238007

Trial 16 summary
Hyperparameters:
use_regularization: False
num_hidden_layers: 4
units_layer_0: 288
activation_layer_0: relu
dropout_0: 0.30000000000000004
optimizer: adam
learning_rate: 0.0004037826117868224
regularizer_type: l2
regularizer_strength: 0.00038492554382474635
units_layer_1: 352

In [19]:
# Get the best model found during the search
best_model = tuner.get_best_models(num_models=1)[0]
print("\nBest model retrieved.")


Best model retrieved.


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
# Evaluate the best model on the validation data
print("\nEvaluating the best model on validation data:")
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Best model validation loss: {loss:.4f}")
print(f"Best model validation accuracy: {accuracy:.4f}")

# Getting the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nBest hyperparameters found:")
print(f"Number of hidden layers: {best_hps.get('num_hidden_layers')}")
for i in range(best_hps.get('num_hidden_layers')):
    print(f"  Layer {i} units: {best_hps.get(f'units_layer_{i}')}")
    print(f"  Layer {i} activation: {best_hps.get(f'activation_layer_{i}')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")


Evaluating the best model on validation data:
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8244 - loss: 0.4324 
Best model validation loss: 0.4504
Best model validation accuracy: 0.8052

Best hyperparameters found:
Number of hidden layers: 2
  Layer 0 units: 128
  Layer 0 activation: tanh
  Layer 1 units: 128
  Layer 1 activation: relu
Learning rate: 0.0007569660497386584


In [21]:
best_model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=200,initial_epoch=21)

Epoch 22/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.7816 - loss: 0.5026 - val_accuracy: 0.7857 - val_loss: 0.4496
Epoch 23/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7768 - loss: 0.4713 - val_accuracy: 0.7987 - val_loss: 0.4570
Epoch 24/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7992 - loss: 0.4654 - val_accuracy: 0.7857 - val_loss: 0.4479
Epoch 25/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7917 - loss: 0.4655 - val_accuracy: 0.7727 - val_loss: 0.4435
Epoch 26/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8064 - loss: 0.4440 - val_accuracy: 0.7792 - val_loss: 0.4458
Epoch 27/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7923 - loss: 0.4634 - val_accuracy: 0.7922 - val_loss: 0.4477
Epoch 28/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7716 - loss: 0.4914 - val_accuracy: 0.7857 - val_loss: 0.4450
Epoch 29/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7654 - loss: 0.4704 - val_accu